In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from Dataset.Economy_Dataset import Economy_Dataset
from Dataset.RNN_Transaction_Dataset import RNN_Transaction_Dataset
from Dataset.NODE_Transaction_Dataset import NODE_Transaction_Dataset

from Model.LSTM import LSTM
from Model.NODE import NODE
from Model.ODEF import *

In [2]:
input_size = 5
hidden_size = 256
output_size = 1

lr = 1e-3
num_epochs = 1

### 부동산 & 경제

In [56]:
transaction_df = pd.read_excel('../데이터/Transaction/transaction_final.xlsx', index_col=0)
economy_df = pd.read_excel('../데이터/Economy/economy_all.xlsx')
economy_df = economy_df['국고채금리']

In [4]:
trainsaction_train_size = int(len(transaction_df)*0.7)
trainsaction_val_size = int(len(transaction_df)*0.3)

transaction_train_dataset = RNN_Transaction_Dataset(transaction_df[:trainsaction_train_size])
transaction_train_loader = DataLoader(transaction_train_dataset, batch_size=2)
transaction_val_dataset = RNN_Transaction_Dataset(transaction_df[trainsaction_train_size:])
transaction_val_loader = DataLoader(transaction_val_dataset, batch_size=2)

economy_train_size = int(len(economy_df)*0.7)
economy_val_size = int(len(economy_df)*0.3)

economy_train_dataset = Economy_Dataset(economy_df[:economy_train_size])
economy_train_loader = DataLoader(economy_train_dataset, batch_size=2)
economy_val_dataset = Economy_Dataset(economy_df[economy_train_size:])
economy_val_loader = DataLoader(economy_val_dataset, batch_size=2)

c:\Users\USER\Desktop\sci\SCI\코드\Dataset\RNN_Transaction_Dataset.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['계약년월'] = pd.to_datetime(data['계약년월'])
c:\Users\USER\Desktop\sci\SCI\코드\Dataset\RNN_Transaction_Dataset.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['계약년월'] = pd.to_datetime(data['계약년월'])
c:\Users\USER\Desktop\sci\SCI\코드\Dataset\Economy_Dataset.py:11: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xe (Triggered 

In [6]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

### 경제 모델

In [16]:
best_val_loss = float('inf') 
for epoch in range(num_epochs + 1):
    model.train()
    for batch_idx, samples in enumerate(economy_train_loader):
        economy_x_train, economy_y_train = samples

        prediction, hidden = model(economy_x_train)
        cost = criterion(prediction, economy_y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_idx, samples in enumerate(economy_val_loader):
            economy_x_val, economy_y_val = samples

            prediction, hidden = model(economy_x_val)
            loss = criterion(prediction, economy_y_val)
            val_loss += loss.item()

    val_loss /= len(economy_val_loader)
    print(f'Epoch {epoch}/{num_epochs}, Training Loss: {cost.item()}, Validation Loss: {val_loss}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '../데이터/Checkpoint/best_rnn_economy_model.pth')

Epoch 0/1, Training Loss: 0.002730900188907981, Validation Loss: 0.3143093644015106
Epoch 1/1, Training Loss: 0.0027428490575402975, Validation Loss: 0.31638226120186774


### 부동산 모델

In [7]:
best_val_loss = float('inf') 
for epoch in range(num_epochs + 1):
    model.train()
    for batch_idx, samples in enumerate(transaction_train_loader):
        dong_x_train, dong_y_train = samples

        prediction, hidden = model(dong_x_train)
        cost = criterion(prediction, dong_y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_idx, samples in enumerate(transaction_val_loader):
            x_val, y_val = samples

            prediction, hidden = model(x_val)
            loss = criterion(prediction, y_val)
            val_loss += loss.item()

    val_loss /= len(transaction_val_loader)
    print(f'Epoch {epoch}/{num_epochs}, Training Loss: {cost.item()}, Validation Loss: {val_loss}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '../데이터/Checkpoint/best_rnn_transaction_model.pth')

Epoch 0/1, Training Loss: 28219.5234375, Validation Loss: 5213844.961625429
Epoch 1/1, Training Loss: 27884.662109375, Validation Loss: 5179137.586078938


### NODE 부동산 돌리기

In [3]:
# 데이터 다운로드
batch_size = 2
transaction_df = pd.read_excel('../데이터/Transaction/transaction_final.xlsx', index_col=0)

train_dataset = NODE_Transaction_Dataset(transaction_df)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

In [4]:
for x,y,z,w in train_loader:
      print("X 크기 : {}".format(x.shape))
      print("Y 크기 : {}".format(y.shape))
      print("Z 크기 : {}".format(z.shape))
      print("W 크기 : {}".format(w.shape))
      break

X 크기 : torch.Size([2, 5])
Y 크기 : torch.Size([2, 5])
Z 크기 : torch.Size([2, 1])
W 크기 : torch.Size([2, 1])


In [5]:
for x,y,z,w in train_loader:
      print("X 크기 : {}".format(x.dtype))
      print("Y 크기 : {}".format(y.dtype))
      print("Z 크기 : {}".format(z.dtype))
      print("W 크기 : {}".format(w.dtype))
      break

X 크기 : torch.float32
Y 크기 : torch.float32
Z 크기 : torch.float32
W 크기 : torch.float32


In [6]:
# 데이터 & 모델에 device 붙임!!!
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # 윈도우 gpu
# device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu') # 맥 gpu
print(f'{device} is available')

model = NODE(output_dim=1,hidden_dim=256,latent_dim=64).to(device)

print('작동하는지 실험')
basic_data = torch.rand((5,2,1)).to(device)  # window_size, batch_size, 1
time = torch.FloatTensor([[1,2,3,6,10,12],[1,3,5,8,10,12]]).reshape(6,2,1).to(device) # window_size, batch_size, 1
data = model(basic_data,time)
print(data)
print(data[0].shape)

cpu is available
작동하는지 실험
(tensor([[[0.3200],
         [0.3020]],

        [[0.3165],
         [0.3053]],

        [[0.3074],
         [0.3091]],

        [[0.3381],
         [0.3102]],

        [[0.3129],
         [0.3069]]], grad_fn=<SliceBackward0>), tensor([[[-1.9246,  0.9916,  0.5816,  ..., -0.0953, -0.4212,  0.2555],
         [-0.3911,  0.1918,  0.4691,  ..., -0.1469, -0.4416,  0.2056]],

        [[-2.3361,  0.9280,  0.5030,  ..., -0.3589, -0.1182, -0.2516],
         [-0.4929,  0.1927, -0.0292,  ...,  0.3396, -0.2586, -0.0658]],

        [[-3.3343,  0.8663,  0.3372,  ..., -1.4191,  0.6670, -1.6950],
         [-0.4960,  0.2321, -0.2474,  ...,  0.5994, -0.1838, -0.2525]],

        [[ 0.8072,  1.3306,  1.7080,  ..., -0.8884, -1.7589,  2.7519],
         [ 0.6183,  0.7583,  1.9381,  ..., -1.3055, -0.8485,  0.8296]],

        [[-2.7293,  0.8898,  0.4417,  ..., -0.7159,  0.1923, -0.7784],
         [-0.5078,  0.2105, -0.1448,  ...,  0.4671, -0.2156, -0.1514]],

        [[ 1.2445,  1.3671

In [7]:
window_size = 5
noise_std = 0.02
optim = torch.optim.Adam(model.parameters(), betas=(0.9, 0.999), lr=0.001)

num_epochs=50

In [8]:
best_train_loss = float('inf') 
for epoch in range(num_epochs + 1):
    print('epoch : ',epoch)
    losses = []
    model.train()
    for batch_idx, samples in enumerate(train_loader):
        tran_x, time_x, tran_y, time_y = samples
        
        t = torch.cat((time_x,time_y),dim=1)  
        tran_x = tran_x.transpose(0,1).unsqueeze(2).to(device)
        t = t.transpose(0,1).unsqueeze(2).to(device)
        
        x_p, _, z, z_mean, z_log_var, pred = model(tran_x, t)
        kl_loss = -0.5 * torch.sum(1 + z_log_var - z_mean**2 - torch.exp(z_log_var), -1)
        loss = 0.5 * ((tran_x-x_p)**2).sum(-1).sum(0) / noise_std**2 + kl_loss
        loss = torch.mean(loss)
        loss /= window_size
        optim.zero_grad()   ############### 여기서 second 오류가 나는지 확인!!!!
        loss.backward()
        optim.step()
        losses.append(loss.item())
        
        if loss < best_train_loss:
            best_train_loss = loss
            torch.save(model.state_dict(), "../데이터/checkpoint/best_ODE_transaction_model.pth")
        
        print('loss : {}, best loss : {}'.format(loss, best_train_loss))
    print('-----------------------------------------------------')

epoch :  0
loss : 11219012608.0, best loss : 11219012608.0
loss : 13275686912.0, best loss : 11219012608.0
loss : 13226204160.0, best loss : 11219012608.0
loss : 11485132800.0, best loss : 11219012608.0
loss : 8940823552.0, best loss : 8940823552.0
loss : 7942053888.0, best loss : 7942053888.0
loss : 8093862912.0, best loss : 7942053888.0
loss : 9578081280.0, best loss : 7942053888.0
loss : 11440328704.0, best loss : 7942053888.0
loss : 13386883072.0, best loss : 7942053888.0
loss : 15193017344.0, best loss : 7942053888.0
loss : 1.5491499810317402e+19, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : na

KeyboardInterrupt: 

### ODE 실험

In [48]:
import math
import numpy as np
from IPython.display import clear_output


import torch
from torch import Tensor
from torch import nn
from torch.nn  import functional as F 
from torch.autograd import Variable

use_cuda = torch.cuda.is_available()

In [49]:
class ODEF(nn.Module):
    def forward_with_grad(self, z, t, grad_outputs):
        """Compute f and a df/dz, a df/dp, a df/dt"""
        batch_size = z.shape[0]

        out = self.forward(z, t)

        a = grad_outputs
        adfdz, adfdt, *adfdp = torch.autograd.grad(
            (out,), (z, t) + tuple(self.parameters()), grad_outputs=(a),
            allow_unused=True, retain_graph=True
        )
        # grad method automatically sums gradients for batch items, we have to expand them back 
        if adfdp is not None:
            adfdp = torch.cat([p_grad.flatten() for p_grad in adfdp]).unsqueeze(0)
            adfdp = adfdp.expand(batch_size, -1) / batch_size
        if adfdt is not None:
            adfdt = adfdt.expand(batch_size, 1) / batch_size
        return out, adfdz, adfdt, adfdp

    def flatten_parameters(self):
        p_shapes = []
        flat_parameters = []
        for p in self.parameters():
            p_shapes.append(p.size())
            flat_parameters.append(p.flatten())
        return torch.cat(flat_parameters)
    
class LinearODEF(ODEF):
    def __init__(self, W):
        super(LinearODEF, self).__init__()
        self.lin = nn.Linear(2, 2, bias=False)
        self.lin.weight = nn.Parameter(W)

    def forward(self, x, t):
        return self.lin(x)
    
class NeuralODE(nn.Module):
    def __init__(self, func):
        super(NeuralODE, self).__init__()
        assert isinstance(func, ODEF)
        self.func = func

    def forward(self, z0, t=Tensor([0., 1.]), return_whole_sequence=False):
        t = t.to(z0)
        z = ODEAdjoint.apply(z0, t, self.func.flatten_parameters(), self.func)
        if return_whole_sequence:
            return z
        else:
            return z[-1]

In [50]:
class ODEAdjoint(torch.autograd.Function):
    @staticmethod
    def forward(ctx, z0, t, flat_parameters, func):
        assert isinstance(func, ODEF)
        bs, *z_shape = z0.size()
        time_len = t.size(0)

        with torch.no_grad():
            z = torch.zeros(time_len, bs, *z_shape).to(z0)
            z[0] = z0
            for i_t in range(time_len - 1):
                z0 = ode_solve(z0, t[i_t], t[i_t+1], func)
                z[i_t+1] = z0

        ctx.func = func
        ctx.save_for_backward(t, z.clone(), flat_parameters)
        return z

    @staticmethod
    def backward(ctx, dLdz):
        """
        dLdz shape: time_len, batch_size, *z_shape
        """
        func = ctx.func
        t, z, flat_parameters = ctx.saved_tensors
        time_len, bs, *z_shape = z.size()
        n_dim = np.prod(z_shape)
        n_params = flat_parameters.size(0)

        # Dynamics of augmented system to be calculated backwards in time
        def augmented_dynamics(aug_z_i, t_i):
            """
            tensors here are temporal slices
            t_i - is tensor with size: bs, 1
            aug_z_i - is tensor with size: bs, n_dim*2 + n_params + 1
            """
            z_i, a = aug_z_i[:, :n_dim], aug_z_i[:, n_dim:2*n_dim]  # ignore parameters and time

            # Unflatten z and a
            z_i = z_i.view(bs, *z_shape)
            a = a.view(bs, *z_shape)
            with torch.set_grad_enabled(True):
                t_i = t_i.detach().requires_grad_(True)
                z_i = z_i.detach().requires_grad_(True)
                func_eval, adfdz, adfdt, adfdp = func.forward_with_grad(z_i, t_i, grad_outputs=a)  # bs, *z_shape
                adfdz = adfdz.to(z_i) if adfdz is not None else torch.zeros(bs, *z_shape).to(z_i)
                adfdp = adfdp.to(z_i) if adfdp is not None else torch.zeros(bs, n_params).to(z_i)
                adfdt = adfdt.to(z_i) if adfdt is not None else torch.zeros(bs, 1).to(z_i)

            # Flatten f and adfdz
            func_eval = func_eval.view(bs, n_dim)
            adfdz = adfdz.view(bs, n_dim) 
            return torch.cat((func_eval, -adfdz, -adfdp, -adfdt), dim=1)

        dLdz = dLdz.view(time_len, bs, n_dim)  # flatten dLdz for convenience
        with torch.no_grad():
            ## Create placeholders for output gradients
            # Prev computed backwards adjoints to be adjusted by direct gradients
            adj_z = torch.zeros(bs, n_dim).to(dLdz)
            adj_p = torch.zeros(bs, n_params).to(dLdz)
            # In contrast to z and p we need to return gradients for all times
            adj_t = torch.zeros(time_len, bs, 1).to(dLdz)

            for i_t in range(time_len-1, 0, -1):
                z_i = z[i_t]
                t_i = t[i_t]
                f_i = func(z_i, t_i).view(bs, n_dim)

                # Compute direct gradients
                dLdz_i = dLdz[i_t]
                dLdt_i = torch.bmm(torch.transpose(dLdz_i.unsqueeze(-1), 1, 2), f_i.unsqueeze(-1))[:, 0]

                # Adjusting adjoints with direct gradients
                adj_z += dLdz_i
                adj_t[i_t] = adj_t[i_t] - dLdt_i

                # Pack augmented variable
                aug_z = torch.cat((z_i.view(bs, n_dim), adj_z, torch.zeros(bs, n_params).to(z), adj_t[i_t]), dim=-1)

                # Solve augmented system backwards
                aug_ans = ode_solve(aug_z, t_i, t[i_t-1], augmented_dynamics)

                # Unpack solved backwards augmented system
                adj_z[:] = aug_ans[:, n_dim:2*n_dim]
                adj_p[:] += aug_ans[:, 2*n_dim:2*n_dim + n_params]
                adj_t[i_t-1] = aug_ans[:, 2*n_dim + n_params:]

                del aug_z, aug_ans

            ## Adjust 0 time adjoint with direct gradients
            # Compute direct gradients 
            dLdz_0 = dLdz[0]
            dLdt_0 = torch.bmm(torch.transpose(dLdz_0.unsqueeze(-1), 1, 2), f_i.unsqueeze(-1))[:, 0]

            # Adjust adjoints
            adj_z += dLdz_0
            adj_t[0] = adj_t[0] - dLdt_0
        return adj_z.view(bs, *z_shape), adj_t, adj_p, None

In [51]:
def ode_solve(z0, t0, t1, f):
    """
    Simplest Euler ODE initial value solver
    """
    h_max = 0.05
    n_steps = math.ceil((abs(t1 - t0)/h_max).max().item())

    h = (t1 - t0)/n_steps
    t = t0
    z = z0

    for i_step in range(n_steps):
        z = z + h * f(z, t)
        t = t + h
    return z

In [52]:
import numpy.random as npr

def gen_batch(batch_size, n_sample=100):
    n_batches = samp_trajs.shape[1] // batch_size
    time_len = samp_trajs.shape[0]
    n_sample = min(n_sample, time_len)
    for i in range(n_batches):
        if n_sample > 0:
            t0_idx = npr.multinomial(1, [1. / (time_len - n_sample)] * (time_len - n_sample))
            t0_idx = np.argmax(t0_idx)
            tM_idx = t0_idx + n_sample
        else:
            t0_idx = 0
            tM_idx = time_len

        frm, to = batch_size*i, batch_size*(i+1)
        yield samp_trajs[t0_idx:tM_idx, frm:to], samp_ts[t0_idx:tM_idx, frm:to]

In [53]:
t_max = 6.29*5
n_points = 200
noise_std = 0.02

num_spirals = 1000

index_np = np.arange(0, n_points, 1, dtype=np.int)
index_np = np.hstack([index_np[:, None]])
times_np = np.linspace(0, t_max, num=n_points)
times_np = np.hstack([times_np[:, None]] * num_spirals)
times = torch.from_numpy(times_np[:, :, None]).to(torch.float32)

# Generate random spirals parameters
normal01 = torch.distributions.Normal(0, 1.0)

x0 = Variable(normal01.sample((num_spirals, 2))) * 2.0  

W11 = -0.1 * normal01.sample((num_spirals,)).abs() - 0.05
W22 = -0.1 * normal01.sample((num_spirals,)).abs() - 0.05
W21 = -1.0 * normal01.sample((num_spirals,)).abs()
W12 =  1.0 * normal01.sample((num_spirals,)).abs()

xs_list = []
for i in range(num_spirals):
    if i % 2 == 1: #  Make it counter-clockwise
        W21, W12 = W12, W21

    func = LinearODEF(Tensor([[W11[i], W12[i]], [W21[i], W22[i]]]))
    ode = NeuralODE(func)

    xs = ode(x0[i:i+1], times[:, i:i+1], return_whole_sequence=True)
    xs_list.append(xs)


orig_trajs = torch.cat(xs_list, dim=1).detach()
samp_trajs = orig_trajs + torch.randn_like(orig_trajs) * noise_std
samp_ts = times

C:\Users\hkyoo\AppData\Local\Temp\ipykernel_27012\2661987758.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  index_np = np.arange(0, n_points, 1, dtype=np.int)


In [54]:
preload = False
n_epochs = 20000
batch_size = 100

plot_traj_idx = 1
plot_traj = orig_trajs[:, plot_traj_idx:plot_traj_idx+1]
plot_obs = samp_trajs[:, plot_traj_idx:plot_traj_idx+1]
plot_ts = samp_ts[:, plot_traj_idx:plot_traj_idx+1]

for epoch_idx in range(n_epochs):
    losses = []
    train_iter = gen_batch(batch_size)
    for x, t in train_iter:

        max_len = np.random.choice([30, 50, 100])
        permutation = np.random.permutation(t.shape[0])
        np.random.shuffle(permutation)
        permutation = np.sort(permutation[:max_len])

        x, t = x[permutation], t[permutation]
        
        break
    break

In [55]:
x.shape

torch.Size([30, 100, 2])

In [56]:
t.shape

torch.Size([30, 100, 1])

In [57]:
model = NODE(output_dim=2,hidden_dim=256,latent_dim=64).to(device)

In [63]:
g = nn.GRU(3,256)
t[1:] = t[:-1] - t[1:]
t[0] = 0.
xt = torch.cat((x, t), dim=-1)

_, h0 = g(xt.flip((0,)))  # Reversed

In [64]:
xt.flip((0,)).shape

torch.Size([30, 100, 3])

In [58]:
model(x,t)

RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 30 but got size 29 for tensor number 1 in the list.